In [ ]:
import pymc3 as pm
import matplotlib.pyplot as plt
import numpy as np
from data import load_finches_2012, load_finches_1975
from utils import ECDF
import arviz as az

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In this notebook, I would like to write an estimation model for beak shape. 

In [ ]:
df12 = load_finches_2012()
df12['shape'] = df12['beak_depth'] / df12['beak_length']

df12 = df12[df12['species'] != 'unknown']
df75 = load_finches_1975()

df = df12  # convenient alias

In [ ]:
df12.head(5)

In [ ]:
fortis_idx = df[df['species'] == 'fortis'].index
scandens_idx = df[df['species'] == 'scandens'].index

# Model : Naive Division of Posteriors

- Estimate posterior for depth and length independently.
- Use posterior samples to estimate distribution for shape.

In [ ]:
# Mega-model incorporating shape as well. 
# We will also analyze the SD in addition to the mean.

with pm.Model() as beak_model:
    # SD can only be positive, therefore it is reasonable to constrain to >0
    # Likewise for betas.
    sd_hyper = pm.HalfCauchy('sd_hyper', beta=100, shape=(2,))
    beta_hyper = pm.HalfCauchy('beta_hyper', beta=100, shape=(2,))
    
    # Beaks cannot be of "negative" mean, therefore, HalfNormal is 
    # a reasonable, constrained prior.
    mean_depth = pm.HalfNormal('mean_depth', sd=sd_hyper[0], shape=(2,))
    sd_depth = pm.HalfCauchy('sd_depth', beta=beta_hyper[0], shape=(2,))
    
    mean_length = pm.HalfNormal('mean_length', sd=sd_hyper[1], shape=(2,))
    sd_length = pm.HalfCauchy('sd_length', beta=beta_hyper[1], shape=(2,))

    nu = pm.Exponential('nu', lam=1/29.) + 1
    
    # Define the likelihood distribution for the data.
    depth = pm.StudentT('beak_depth', 
                        nu=nu,
                        mu=mean_depth[df['species_enc']], 
                        sd=sd_depth[df['species_enc']], 
                        observed=df['beak_depth'])
    
    length = pm.StudentT('beak_length',
                         nu=nu,
                         mu=mean_length[df['species_enc']],
                         sd=sd_length[df['species_enc']],
                         observed=df['beak_length'])

In [ ]:
with beak_model:
    trace = pm.sample(2000)

In [ ]:
az.plot_trace(trace, var_names=['mean_length', 'mean_depth'])

In [ ]:
pm.traceplot(trace, varnames=['sd_length', 'sd_depth'])

In [ ]:
samples = pm.sample_ppc(trace, model=beak_model)
samples

PPC check for Fortis

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122, sharex=ax1)

def plot_ppc_data(samples, df, idxs, column, ax):
    x, y = ECDF(samples[column][:, idxs].flatten())
    ax.plot(x, y, label='ppc')
    x, y = ECDF(df.iloc[idxs][column])
    ax.plot(x, y, label='data')
    ax.set_xlabel(column)
    ax.set_ylabel('cumulative fraction')
    return ax

ax1 = plot_ppc_data(samples, df, fortis_idx, 'beak_depth', ax1)
ax2 = plot_ppc_data(samples, df, fortis_idx, 'beak_length', ax2)

fig.suptitle('Fortis')
plt.tight_layout()

PPC check for Scandens

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122, sharex=ax1)

ax1 = plot_ppc_data(samples, df, scandens_idx, 'beak_depth', ax1)
ax2 = plot_ppc_data(samples, df, scandens_idx, 'beak_length', ax2)

fig.suptitle('Scandens')
plt.tight_layout()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
x, y = ECDF((samples['beak_depth'][:, fortis_idx] / samples['beak_length'][:, fortis_idx]).flatten())
ax.plot(x, y)
x, y = ECDF(df.loc[fortis_idx, 'shape'])
ax.plot(x, y)

Ok, looks like this is not the right model. Dividing PPC samples is definitely not the right approach here.

Maybe jointly modelling the observed beak and length distributions is the right thing to do?

In [ ]:
fig = plt.figure(figsize=(12, 4))

def plot_length_depth_scatter(df, idxs, title, ax):
    ax.scatter(df.iloc[idxs]['beak_length'], df.iloc[idxs]['beak_depth'])
    ax.set_xlabel('beak_length')
    ax.set_ylabel('beak_depth')
    ax.set_title(title)
    return ax


ax1 = fig.add_subplot(121)
ax1 = plot_length_depth_scatter(df, scandens_idx, 'scandens', ax1)

ax2 = fig.add_subplot(122, sharex=ax1, sharey=ax1)
ax2 = plot_length_depth_scatter(df, fortis_idx, 'fortis', ax2)


# Model: Joint Distribution

Going to try a new model: we explicity model depth and length jointly, as a multivariate gaussian.

In [ ]:
with pm.Model() as mv_beaks:  # multivariate beak model
    packed_L = pm.LKJCholeskyCov('packed_L', n=2,
                             eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
    L = pm.expand_packed_triangular(2, packed_L)
    sigma = pm.Deterministic('sigma', L.dot(L.T))

    mu = pm.HalfNormal('mu', sd=20, shape=(2,))
    
    like = pm.MvNormal('like', mu=mu, cov=sigma, observed=df.iloc[scandens_idx][['beak_depth', 'beak_length']].values)

In [ ]:
with mv_beaks:
    trace_mv = pm.sample(2000, njobs=1)

In [ ]:
az.plot_trace(trace_mv)

In [ ]:
az.plot_forest(trace_mv, var_names=['sigma'])

In [ ]:
az.plot_forest(trace_mv, var_names=['mu'])

In [ ]:
samples_mv = pm.sample_ppc(trace_mv, model=mv_beaks)

In [ ]:
samples_mv['like'][:, 0]  # beak_depth
samples_mv['like'][:, 1]  # beak_length

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

x, y = ECDF(samples_mv['like'][:, 0])
ax1.plot(x, y, label='ppc')
x, y = ECDF(df.iloc[scandens_idx]['beak_depth'])
ax1.plot(x, y, label='data')
ax1.set_title('beak depth')
ax1.legend()

x, y = ECDF(samples_mv['like'][:, 1])
ax2.plot(x, y, label='ppc')
x, y = ECDF(df.iloc[scandens_idx]['beak_length'])
ax2.plot(x, y, label='data')
ax2.set_title('beak length')
ax2.legend()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

x, y = ECDF(trace_mv['sigma'][:, 0, 1])
ax.plot(x, y, label='samples')
x, y = ECDF(df.iloc[scandens_idx]['shape'])
ax.plot(x, y)

# Model: Regress Depth on Length

Maybe the right way to compute shape is to regress depth on length, and compute the slope. After all, that's all that depth/length really is.

We will assume a model: $y=mx$, no intercept.

In [ ]:
with pm.Model() as shape_model:
    shape = pm.Normal('shape', mu=0, sd=100)
    sd = pm.HalfCauchy('sd', beta=100)
    
    mu = shape * df.iloc[scandens_idx]['beak_length'].values
    
    like = pm.Normal('like', mu=mu, sd=sd, observed=df.iloc[scandens_idx]['beak_depth'].values)

In [ ]:
with shape_model:
    trace_shape = pm.sample(2000)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

x, y = ECDF(trace_shape['shape'])
ax.plot(x, y, label='sample')
x, y = ECDF(df.iloc[scandens_idx]['shape'].values)
ax.plot(x, y, label='data')
ax.legend()

I have the model mis-specified - I get the posterior distribution over the slope, but not the distribution of shapes. I guess shapes and slopes are kind of different. 

Let's try just estimating shape directly.

# Model: Estimate on Shape Parameter

In [ ]:
with pm.Model() as shape_model:
    mu = pm.HalfNormal('mu', sd=100)
    sd = pm.HalfCauchy('sd', beta=100)
    
    like = pm.Normal('shape', mu=mu, sd=sd, observed=df.iloc[scandens_idx]['shape'].values)

In [ ]:
with shape_model:
    trace = pm.sample(2000)

In [ ]:
samples = pm.sample_ppc(trace, model=shape_model)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

x, y = ECDF(samples['shape'].flatten())
ax.plot(x, y, label='samples')
x, y = ECDF(df.iloc[scandens_idx]['shape'])
ax.plot(x, y, label='data')
ax.legend()

As it turns out, the simplest model is the best fitting one...